In [1]:
import gc
import os
import cv2
import sys
import json
import tqdm
import time
import mindspore
import random
import sklearn.metrics

from PIL import Image
from pathlib import Path
from functools import partial
from contextlib import contextmanager

import numpy as np
import scipy as sp
import pandas as pd
import mindspore.nn as nn

from mindspore.optim import Adam, SGD
from scipy.special import softmax
from mindspore.utils.data import DataLoader, Dataset
from albumentations import Compose, Normalize, Resize
from mindspore.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import f1_score, accuracy_score, top_k_accuracy_score
from math import radians,cos,sin,asin,sqrt,pi
import mindspore.optim as optim
import mindspore.utils.data as data
import shutil
seed = 2022
random.seed(seed)
np.random.seed(seed)
mindspore.manual_seed(seed)

os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = mindspore.device('cuda' if mindspore.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
all_data1 = pd.read_csv('./all_data_logits/swin_large.csv')
all_data2 = pd.read_csv('./all_data_logits/swin_base.csv')
all_data3 = pd.read_csv('./all_data_logits/eff_b6.csv')
all_data4 = pd.read_csv('./all_data_logits/eff_b7.csv')
all_data5 = pd.read_csv('./all_data_logits/beit.csv')

In [3]:
group_scores2 = all_data5.groupby(['ObservationId']).mean().reset_index()

In [4]:
preds_raw = all_data4.iloc[:,1:]+all_data3.iloc[:,1:]+all_data1.iloc[:,1:]+all_data2.iloc[:,1:] 
scores = pd.concat([all_data1['ObservationId'], preds_raw], axis=1)
group_scores = scores.groupby(['ObservationId']).mean().reset_index()


submit = pd.DataFrame()
# ObservationId,ClassId
submit['ObservationId'] = group_scores['ObservationId']
submit['ClassId'] = np.argmax(np.array(group_scores.iloc[:, 1:]+group_scores2.iloc[:, 1:]), axis=1)

In [5]:
test_metadata = pd.read_csv("/home/data3/changhao/Datasets/FGVC2022_Fungi/FungiCLEF2022_test_metadata.csv")
for i in list(test_metadata[test_metadata['Substrate']=='spiders']['ObservationId']):
    print(submit[submit["ObservationId"]==i].index[0])
    submit.loc[submit[submit["ObservationId"]==i].index[0],'ClassId'] = -1


submit.to_csv('final_result.csv', index=False, header=True)

38199
32371
54213
54213
45533
45533
45533
45533
45533
45533
45533
45533
45533
45533
56729
56729
3833
3833
3833
3833
3833
3833
3833
3833
3833
38521


In [6]:
submit[submit.ClassId==-1]

,ObservationId,ClassId
3833,3027625304,-1
32371,3355967414,-1
38199,3380898495,-1
38521,3380901444,-1
45533,3392601328,-1
54213,3410061329,-1
56729,3417635304,-1
